# <strong> Quants SQL
DB 활용 주식분석 (SQL이 보이는 그림책)
https://wikidocs.net/5332


<br>
## <strong>1 SQL 생성/입력/수정/삭제
### <strong>01 INSERT
데이터 입력하기

In [154]:
# SQL 구문을 호충
import sqlite3
con = sqlite3.connect('./sqlite.db')
cursor = con.cursor()
cursor.execute("SELECT  DATE('now');")
cursor.fetchall()

[('2017-12-27',)]

In [89]:
# 1. 컬럼을 생략하고 데이터 입력
# INSERT INTO 테이블 VALUES (1열, 2열, 3열)
cursor.execute("CREATE TABLE tbl_tea (code int, name str, price int);")
cursor.execute("INSERT INTO tbl_tea VALUES (1, '녹차', 7000);")
cursor.execute("INSERT INTO tbl_tea VALUES (2, '홍차', 8500);")
con.commit()

In [91]:
cursor.execute("SELECT * FROM tbl_tea;")
cursor.fetchall()

[(1, '녹차', 7000), (2, '홍차', 8500)]

In [92]:
# 2. 특정 컬럼에만(name, code) 값을 입력하기 
# PRICE 에는 None/ NULL/ NA 가 입력된다
cursor.execute("""INSERT INTO   tbl_tea (name,  code) 
                         VALUES         ('구기자차', 3);""")
cursor.execute("SELECT * FROM tbl_tea;")
cursor.fetchall()

[(1, '녹차', 7000), (2, '홍차', 8500), (3, '구기자차', None)]

In [155]:
# 3. 테이블 복사
# 기존의 테이블 내용을 새로운 테이블로 복붙할 경우
cursor.execute("""CREATE TABLE   tbl_petlist (id int, name str);""")
cursor.execute("""INSERT INTO    tbl_petlist (id, name)    /* 입력대상 테이블 */ 
                         SELECT              code, name    /* 대상 필드 설정 */
                         FROM    tbl_tea;                  /* 복사할 원본 테이블*/""") 
cursor.execute("SELECT * FROM tbl_petlist;")
cursor.fetchall()

[(1, '녹차'), (2, '홍차')]

In [156]:
con.commit()
con.close()

### <strong>02 UPDATE
등록된 데이터를 수정

In [96]:
# UPDATE() 하나의 값 수정
cursor.execute("""UPDATE  tbl_petlist  
                          SET name = '고양이'  
                          WHERE id = 2;""")
cursor.execute("SELECT * FROM tbl_petlist;")
cursor.fetchall()

[(1, '녹차'), (2, '고양이'), (3, '구기자차')]

In [98]:
# UPDATE() 여러 값 수정하기
cursor.execute("""UPDATE  tbl_petlist  
                          SET name = '마르티즈'  
                          WHERE id >= 3;""")
cursor.execute("SELECT * FROM tbl_petlist;")
cursor.fetchall()

[(1, '페키니즈'), (2, '고양이'), (3, '마르티즈')]

### <strong>03 DELETE
DELETE 등록 데이터 삭제

In [99]:
# UPDATE() 여러 값 수정하기
cursor.execute("""DELETE FROM  tbl_petlist  
                         WHERE id = 3;""")  # 대상을 제한
cursor.execute("SELECT * FROM tbl_petlist;")
cursor.fetchall()

[(1, '페키니즈'), (2, '고양이')]

In [100]:
# UPDATE() 모든 데이터 삭제
cursor.execute("""DELETE FROM tbl_petlist;""") # 테이블 삭제
cursor.execute("SELECT * FROM tbl_petlist;")
cursor.fetchall()

[]

<br>
## <strong>2 SQL 기본조작
### <strong>01 SELECT 조건
컬럼, 테이블의 모든 데이터 가져오기

In [8]:
# 특정 컬럼만 추출 
import sqlite3
import pandas as pd
db = './data/krxStock.db'
con = sqlite3.connect(db)

# 1. 전체 테이블 불러오기
df = pd.read_sql("SELECT  *  FROM   stock_trader ;",con)
df.head(2)

,Date,Code,개인,외국인계,기관계,증권,투신,은행,종금,보험,기금,기타
0,2016/05/18,047770.KQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016/05/18,014200.KQ,4678.0,-756.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5100.0


In [11]:
# 2. Code 컬럼 추출
df = pd.read_sql("""SELECT   Code, 개인, 외국인계    
                    FROM     stock_trader ;""", con)
df.head(2)

,Code,개인,외국인계
0,047770.KQ,0.0,0.0
1,014200.KQ,4678.0,-756.0


In [12]:
# 3. 데이터 정렬  (ASC : 오름차순 , DESC : 내림차순)
df = pd.read_sql("""SELECT     Date, Code, 개인, 외국인계    
                    FROM       stock_trader
                    ORDER BY   외국인계  DESC ;""", con)
df.head(3)

,Date,Code,개인,외국인계
0,2017/07/07,105560.KS,254904.0,6731165.0
1,2017/04/05,044180.KQ,-4239124.0,4209824.0
2,2016/08/03,003450.KS,-6855390.0,4186668.0


In [13]:
# 4. Date as 날짜 : 컬럼명을 바꿔서 출력
df = pd.read_sql("""SELECT    Date as 날짜, Code, 개인, 외국인계   
                    FROM      stock_trader  
                    ORDER BY  외국인계  DESC ;""", con)
df.head(2)

,날짜,Code,개인,외국인계
0,2017/07/07,105560.KS,254904.0,6731165.0
1,2017/04/05,044180.KQ,-4239124.0,4209824.0


### <strong>02 WHERE 조건
조건을 사용하여 특정 데이터를 호출

In [14]:
# 1. WHERE   Date = '2017/12/26'
df = pd.read_sql("""SELECT  Date, Code, 개인, 외국인계   
                    FROM    stock_trader  
                    WHERE   Date = '2017/12/26' ;""", con)
df.head(3)

,Date,Code,개인,외국인계
0,2017/12/26,269620.KQ,2556465,-458298
1,2017/12/26,189980.KQ,1051,-1092
2,2017/12/26,033180.KS,-37468,39413


In [15]:
# 2. WHERE  개인 >= 0  ORDER BY 외국인계  DESC
df = pd.read_sql("""SELECT  * 
                    FROM    stock_trader  
                    WHERE   개인 >= 0  
                        ORDER BY  외국인계  DESC ;""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계,증권,투신,은행,종금,보험,기금,기타
0,2017/07/07,105560.KS,254904,6731165,733760,244441,206846,3090,0.0,123542.0,21535.0,-7720355.0
1,2016/10/06,044180.KQ,1075472,2682607,-4424784,-4424784,0,0,0.0,0.0,0.0,603148.0
2,2016/06/09,088350.KS,198370,1946050,-2140202,73166,-1626278,0,0.0,-406093.0,14522.0,20.0


In [16]:
# 3. WHERE  Date >= '2017/12/22'  ORDER BY 외국인계  DESC
# 날짜를 기준으로 데이터 추출 (더 빠르더라)
df = pd.read_sql("""SELECT   *   
                    FROM     stock_trader  
                    WHERE    Date >= '2017/12/22'  
                        ORDER BY  외국인계  DESC ;""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계,증권,투신,은행,종금,보험,기금,기타
0,2017/12/22,004410.KS,-1321667,1336672,593,593,0,0,0.0,0.0,0.0,99.0
1,2017/12/22,044180.KQ,-492579,995756,3,3,0,0,0.0,0.0,0.0,-500000.0
2,2017/12/26,035890.KQ,-1436113,634683,545341,372515,173820,0,0.0,3254.0,0.0,264522.0


### <strong>03 산술 연산자 
수치연산을 활용한다 : +, -, *, /, %(나머지)

In [9]:
# SELECT  (외국인계+기관계) as 세력  : 컬럼의 수식 활용
df = pd.read_sql("""SELECT   Date, Code, 개인, 외국인계, 
                             (기관계+증권+투신+은행+기타) as 국내계 
                    FROM     stock_trader 
                    WHERE    Date >= '2017/10/22';""", con)
df.head(3)

,Date,Code,개인,외국인계,국내계
0,2017/10/23,002840.KS,-3800.0,0.0,3800.0
1,2017/10/23,024850.KQ,21173.0,-21201.0,56.0
2,2017/10/23,047050.KS,153402.0,-93974.0,-61240.0


### <strong>04 비교, 논리 연산자
비교 (>, <, >=, <=),  논리(and, or, not, in)

In [20]:
# SELECT  WHERE 조건1 AND 조건2  ;
df = pd.read_sql("""SELECT  *  
                    FROM    stock_trader 
                    WHERE   Date   >= '2017/10/22' AND
                            외국인계 >=  0  ;""",   con)
df.head(3)

,Date,Code,개인,외국인계,기관계,증권,투신,은행,종금,보험,기금,기타
0,2017/10/23,002840.KS,-3800,0,0,0,0,0,0.0,0.0,0.0,3800.0
1,2017/10/23,032960.KQ,-3290,0,42,42,0,0,0.0,0.0,0.0,3248.0
2,2017/10/23,065650.KQ,-125106,48545,0,0,0,0,0.0,0.0,0.0,72000.0


In [26]:
# SELECT  WHERE 조건1 or 조건2  ;
df = pd.read_sql("""SELECT   Date, Code, 개인, 외국인계, 기관계  
                    FROM     stock_trader 
                    WHERE    Date >= '2017/10/22' AND 
                             ((외국인계 % 10) >= 5) ;""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계
0,2017/10/23,065650.KQ,-125106,48545,0
1,2017/10/23,002150.KS,-1793,1925,-17
2,2017/10/23,052400.KQ,2561,2755,-1442


### <strong>05 문자처리 연산자
문자 연결 및 비교
1. <strong>'%'</strong> : 시작<strong>%</strong>, <strong>%</strong>끝, <strong>%</strong>중간<strong>%</strong>
1. <strong>_</strong> : 임의의 1개 문자

In [35]:
# 1. Date || Code : 두개의 문자열을 합친다
df = pd.read_sql("""SELECT   Date || Code  
                    FROM     stock_trader 
                    WHERE    Date >= '2017/10/22' ;""", con)
df.head(3)

,Date || Code
0,2017/10/23002840.KS
1,2017/10/23024850.KQ
2,2017/10/23047050.KS


In [34]:
# 2. Code  LIKE '000%'  : '000'으로 시작하는 데이터 추출
df = pd.read_sql("""SELECT  Date, Code      
                    FROM    stock_trader 
                    WHERE   Date  >=   '2017/10/22' AND  
                            Code  LIKE '000%';""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계
0,2017/10/23,000910.KS,3185,-3161,-24
1,2017/10/23,000860.KS,975,353,-1324
2,2017/10/23,000760.KS,1,0,-1


In [36]:
# 3. Code  LIKE '%KQ' :   KQ로 끝나는 문자열
df = pd.read_sql("""SELECT  Date, Code      
                    FROM    stock_trader 
                    WHERE   Date  >=   '2017/10/22' AND  
                            Code  LIKE '%KQ';""", con)
df.head(3)

,Date,Code
0,2017/10/23,024850.KQ
1,2017/10/23,256940.KQ
2,2017/10/23,032960.KQ


### <strong>06 기타 연산자
1. <strong>'%'</strong> : 시작<strong>%</strong>, <strong>%</strong>끝, <strong>%</strong>중간<strong>%</strong>
1. <strong>_</strong> : 임의의 1개 문자

In [41]:
# WHERE   외국인계  BETWEEN  100  AND  100000
df = pd.read_sql("""SELECT   Date, Code, 개인, 외국인계, 기관계   
                    FROM     stock_trader 
                    WHERE    Date    >=  '2017/10/22' AND  
                             외국인계  BETWEEN 100 AND 100000;""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계
0,2017/10/23,065650.KQ,-125106,48545,0
1,2017/10/23,036090.KQ,-27289,24410,72
2,2017/10/23,072020.KQ,-3431,3431,0


In [43]:
# WHERE  WHERE 외국인계 IS NULL / IS NOT NULL  : NULL 값을 확인한다
df = pd.read_sql("""SELECT  Date, Code, 개인, 외국인계, 기관계   
                    FROM    stock_trader 
                    WHERE   외국인계  IS  NULL;""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계
0,2017/10/24,0126,None,None,None


In [44]:
# WHERE  Code  IN/ NOT IN  ('005930.KS', '000390.KS') : 특정 열의 포함여부를 확인
df = pd.read_sql("""SELECT  Date, Code, 개인, 외국인계, 기관계   
                    FROM    stock_trader 
                    WHERE   Date  >= '2017/10/22' AND  
                            Code  IN ('005930.KS', '000390.KS') ;""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계
0,2017/10/23,005930.KS,5683,30468,-63060
1,2017/10/23,000390.KS,-1935,2775,-840
2,2017/10/24,000390.KS,-4316,4340,-24


<br>
## <strong>3 SQL 함수
### <strong>01 수치/ 문자 함수
소숫점의 연산

In [17]:
# 1. LENGTH() : 데이터 문자열 길이출력 
import sqlite3
conn = sqlite3.connect("./data/krxStock.db")
cur = conn.cursor()
SQL = """SELECT   Date, Code, 
                  LENGTH(개인) AS result    /* LENGTH() : 데이터 문자열 길이출력 */
         FROM     stock_trader;"""
cur.execute(SQL)
for no, row in enumerate(cur.fetchall()):
    if no != 4  : print(row)
    elif no == 4: break

('2016/05/18', '047770.KQ', 1)
('2016/05/18', '014200.KQ', 4)
('2016/05/18', '009780.KQ', 5)
('2016/05/18', '123420.KQ', 5)


In [20]:
# SUBSTR/ SUBSTRING (필드명, 왼쪽에서 시작점,  총 가져올 String수)
SQL = """SELECT   Date, Code,
                  SUBSTR (Code,2,4) as result  /* (객체변수, 시작점, 가져올갯수) */
         FROM     stock_trader; """
cur.execute(SQL)
for no, row in enumerate(cur.fetchall()):
    if no != 4  : print(row)
    elif no == 4: break

('2016/05/18', '047770.KQ', '4777')
('2016/05/18', '014200.KQ', '1420')
('2016/05/18', '009780.KQ', '0978')
('2016/05/18', '123420.KQ', '2342')


In [25]:
# LOWER(소문자로 변환), UPPER(대문자로 변환)
SQL = """SELECT   Date, 
                  LOWER(Code) as pCode   /* LOWER(소문자로 변환), UPPER(대문자로 변환) */
         FROM     stock_trader;"""
cur.execute(SQL)
for no, row in enumerate(cur.fetchall()):
    if no != 4  : print(row)
    elif no == 4: break

('2016/05/18', '047770.kq')
('2016/05/18', '014200.kq')
('2016/05/18', '009780.kq')
('2016/05/18', '123420.kq')


### <strong>02 날짜 함수
날짜를 다루는 함수들
https://sqlite.org/lang_datefunc.html  : DB별 함수들이 다름에 유의

In [2]:
# SQL 구문을 호충
import sqlite3
con = sqlite3.connect('./sqlite.db')
cursor = con.cursor()
cursor.execute("SELECT  DATE('now');")
cursor.fetchall()

[('2017-12-28',)]

In [3]:
# 해당 월의 마지막 날짜를 계산한다 (해당월 다음 월의 1일전의 날짜를 출력)
cursor.execute("""SELECT  DATE('now', 'start of month',
                               '+1 month',
                               '-1 day') ;""")
cursor.fetchall()

[('2017-12-31',)]

In [4]:
cursor.execute("""SELECT  strftime('%s','now') 
                        - strftime('%s','2004-01-01 02:34:56');""" )
cursor.fetchall()

[(441528872,)]

In [5]:
# 10월 (1월 + 9월 의 첫번쨰 화요일 출력
cursor.execute("""SELECT  DATE('now','start of year',
                               '+9 months',
                               'weekday 2'); """)
cursor.fetchall()

[('2017-10-03',)]

In [6]:
# %m   : 월 자료의 연산시
# %m%d : 월일 자료간 연산시 (9월 09일)
# %d   : 일 자료간 연산시
cursor.execute("""SELECT  strftime('%m%d','2011-10-10') 
                        - strftime('%m%d','2012-01-01');""")
cursor.fetchall()

[(909,)]

In [145]:
con.close()

### <strong>03 집합함수
하나의 '필드'에 대한 연산결과를 집합하여 출력

In [71]:
# 1. AVG, SUM, COUNT, MAX, MIN 
import sqlite3
conn = sqlite3.connect("./data/krxStock.db")
cur = conn.cursor()
SQL = """SELECT  Date, Code, 
                 AVG(개인) as average,   /*AVG(필드)   : 필드 값들의 평균 */ 
                 SUM(개인) as total,     /*SUM(필드)   : 필드 값들의 합계 */
                 COUNT(개인) as count    /*COUNT(필드) : 필드 객체 갯수   */
          FROM   stock_trader  
          WHERE  Date >= '2017/10/22';"""
cur.execute(SQL)
cur.fetchall()

[('2017/12/26', '039030.KQ', 1202.990261370832, 110186690, 91594)]

In [76]:
# HAVING(조건) : Group By 에 조건을 추가
SQL = """SELECT    Date, Code, 개인, 외국인계
         FROM      stock_trader  
         GROUP BY  Code   
                   HAVING ( MAX( 외국인계 ) > 4000000 );"""
cur.execute(SQL)
cur.fetchall()

[('2016/08/03', '003450.KS', -6855390, 4186668),
 ('2017/04/05', '044180.KQ', -4239124, 4209824),
 ('2017/07/07', '105560.KS', 254904, 6731165)]

In [78]:
# GROUP BY  Code   HAVING (조건);
# 조건을 만족하는 Code들만 출력
SQL = """SELECT  CAST ('2000-02-02' as DATETIME);""" # DATETIME 포맷으로 변환한다
cur.execute(SQL)
cur.fetchall()

[(2000,)]

In [79]:
# GROUP BY  Code   HAVING (조건);
# 조건을 만족하는 Code들만 출력
SQL = """SELECT  Date  as  moonjay,  
                 CAST(Date  AS  DATETIME)  as nalzz,
                 Code, 개인, 외국인계
         FROM    stock_trader  
         WHERE   Date >= '2017/10/22';"""
cur.execute(SQL)
for no, row in enumerate(cur.fetchall()):
    if no != 4  : print(row)
    elif no == 4: br

('2017/10/23', 2017, '002840.KS', -3800, 0)
('2017/10/23', 2017, '024850.KQ', 21173, -21201)
('2017/10/23', 2017, '047050.KS', 153402, -93974)
('2017/10/23', 2017, '256940.KQ', -99, -300)


<br>
## <strong>4 Sub Query
<strong>서브쿼리</strong> : SELECT 내부에 SELECT를 사용

In [128]:
# 특정 컬럼만 추출 
import sqlite3
import pandas as pd
db = './data/krxStock.db'
con = sqlite3.connect(db)

In [129]:
# 1. WHERE 에서 '서브쿼리문' 사용 : SELECT AVG(외국인계) FROM stock_trader
df = pd.read_sql("""SELECT  Date, Code, 개인, 외국인계, 기관계
                    FROM    stock_trader 
                    WHERE   Date >= '2017/11/22'  AND
                            개인  >= (SELECT  AVG(외국인계) 
                                     FROM    stock_trader) ;""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계
0,2017/11/22,056090.KQ,8428,-3045,-5383
1,2017/11/22,259630.KQ,38228,2413,-8754
2,2017/11/22,196700.KQ,3756,-2959,-780


In [110]:
# 2. HAVING 에서 '서브쿼리문' 사용
# HAVING MIN(기타) < (SELECT AVG(외국인계) FROM stock_trader)
# 기타의 최소값이, 외국인 평균보다 큰 데이터를 CODE별 모음
df = pd.read_sql("""SELECT      Date, Code, 개인, 외국인계, 기관계, MIN(기타) as ETC
                    FROM        stock_trader 
                    GROUP BY    Code
                                HAVING  MIN(기타) < (SELECT  AVG(외국인계) 
                                                    FROM    stock_trader);""", con)
df.head(3)

,Date,Code,개인,외국인계,기관계,ETC
0,2016/08/31,000020.KS,92620,-8611,4309,-88518.0
1,2017/07/21,000030.KS,167963,676830,-346369,-499192.0
2,2017/09/11,000040.KS,280910,102024,-167410,-215624.0


In [125]:
# 3. FROM 절에서 '서브쿼리문' 사용
# FROM (SELECT * FROM  stock_trader  WHERE  외국인계 >=4000000) 에서 데이터 추출
df = pd.read_sql("""SELECT Date, Code, 개인, 외국인계, 기관계
                    FROM   (SELECT  * 
                            FROM    stock_trader  
                            WHERE   외국인계 >=4000000) AS c_price;""", con)
df

,Date,Code,개인,외국인계,기관계
0,2016/08/03,003450.KS,-6855390,4186668,3285520
1,2017/04/05,044180.KQ,-4239124,4209824,114
2,2017/07/07,105560.KS,254904,6731165,733760
3,2017/08/14,044180.KQ,-4187182,4014190,0


In [ ]:
# 4. SUBQUERY를 INSERT/ UPDATE/ DELETE에 응용
# 01. INSERT : tbl_results 쿼리결과를 tbl_advance 에 입력
"""INSERT  INTO  tbl_advance  
   SELECT  * 
   FROM    tbl_results
   WHERE   point1 + point2 >  (SELECT  AVG(point1 + point2)  
                               FROM    tbl_results)"""

In [ ]:
# 02. UPDATE : 
"""UPDATE  tbl_allowance  
   SET     total    = (overtime + travel)
   WHERE   overtime + travel < (SELECT  MAX(overtime) 
                                FROM    tbl_allowance;)"""

In [ ]:
# 03. DELETE : WHERE 조건쿼리 결과를 삭제
"""DELETE  
   FROM    tbl_allowance  
   WHERE   travel > (SELECT  AVG(travel)  
                     FROM    tbl_allowance  
                     WHERE   overtime >= 40000);"""

<br>
## <strong>5 복잡한 SQL 데이터 조작
<strong>JOIN</strong> : Cross/ INNer/ Outer

<img src="http://kirillpavlov.com/images/join-types.png" width=600 align='left'>

### <strong>01 교차조인 : 합집합
<strong>'서브쿼리 결과'</strong>를 사용하여, <strong>'메인쿼리 결과'</strong>를 가져온다

In [136]:
# 원본  CROSS JOIN  대상
con = sqlite3.connect('./sqlite.db')
df = pd.read_sql("""SELECT      *  
                    FROM        tbl_tea  /* 원본 */ 
                    CROSS JOIN  test;    /* 대상 */""", con)
df

,code,name,price,Date,Open,High,Low,Close,Adj Close,Volume
0,1,녹차,7000,2000-02-11 00:00:00,29300.0,30800.0,28350.0,30800.0,22649.746094,332800.0
1,1,녹차,7000,2000-02-14 00:00:00,33000.0,34450.0,28500.0,34450.0,25333.888672,1518600.0
2,2,홍차,8500,2000-02-11 00:00:00,29300.0,30800.0,28350.0,30800.0,22649.746094,332800.0
3,2,홍차,8500,2000-02-14 00:00:00,33000.0,34450.0,28500.0,34450.0,25333.888672,1518600.0


In [133]:
# Simple 테이블 결합
# SELECT   컬럼   FROM   원본 , 대상
df = pd.read_sql("""SELECT   * 
                    FROM     tbl_tea, test;""", con)
df

,code,name,price,Date,Open,High,Low,Close,Adj Close,Volume
0,1,녹차,7000,2000-02-11 00:00:00,29300.0,30800.0,28350.0,30800.0,22649.746094,332800.0
1,1,녹차,7000,2000-02-14 00:00:00,33000.0,34450.0,28500.0,34450.0,25333.888672,1518600.0
2,2,홍차,8500,2000-02-11 00:00:00,29300.0,30800.0,28350.0,30800.0,22649.746094,332800.0
3,2,홍차,8500,2000-02-14 00:00:00,33000.0,34450.0,28500.0,34450.0,25333.888672,1518600.0


### <strong>02 내부조인 : 교집합
<strong>'서브쿼리 결과'</strong>를 사용하여, <strong>'메인쿼리 결과'</strong>를 가져온다

In [161]:
# INNER JOIN 객체간 연결
import pandas as pd
con = sqlite3.connect('./sqlite.db')
df = pd.read_sql("""SELECT  * 
                    FROM    tbl_petlist
                    INNER   JOIN  tbl_tea  
                            ON    id = code ;""", con); df

,id,name,code,name,price
0,1,녹차,1,녹차,7000
1,2,홍차,2,홍차,8500


In [163]:
# INNER JOIN 객체간 연결 2
df = pd.read_sql("""SELECT   * 
                    FROM     tbl_petlist
                    INNER   JOIN  tbl_tea  
                            ON    tbl_tea.code = tbl_petlist.id;""", con); df

,id,name,code,name,price
0,1,녹차,1,녹차,7000
1,2,홍차,2,홍차,8500


### <strong>03 외부 조인
<strong>내부조인과 일치하지 않아도 가져올 수 있다

In [167]:
# LEFT JOIN  : 'tbl_petlist'을 기준으로 조인
# RIGHT JOIN : 'tbl_tea'을 기준으로 조인
# FULL JOIN  : 전체 외부조인
df = pd.read_sql("""SELECT   * 
                    FROM     tbl_petlist
                    LEFT     JOIN  tbl_tea  
                             ON    tbl_tea.code = tbl_petlist.id;""", con) ;df

,id,name,code,name,price
0,1,녹차,1,녹차,7000
1,2,홍차,2,홍차,8500


### <strong>04 WHERE절 외부 조인
<strong>WHERE 절로 JOIN을 활용

In [176]:
df = pd.read_sql("""SELECT   * 
                    FROM     tbl_petlist, tbl_tea  
                    WHERE    tbl_tea.code = tbl_petlist.id;""", con) ;df

,id,name,code,name,price
0,1,녹차,1,녹차,7000
1,2,홍차,2,홍차,8500


### <strong>05 VIEW의 작성
viw_temp2 : VIEW를 제작 (임시 Template를 작성한다)

In [181]:
# 1. VIEW 의 생성
con = sqlite3.connect('./sqlite.db')
cur = con.cursor()
SQL = """CREATE  VIEW  viw_temp2  AS  
         SELECT  name, price  
         FROM    tbl_tea  
         WHERE   code = 1;"""
cur.execute(SQL)
cur.fetchall()  # VIEW를 제작하면 별도로 출력되는 것은 없다

[]

In [ ]:
# 2. VIEW 의 등록/ 수정/ 삭제
SQL_ = """INSERT INTO   viw_temp (name, price) 
                 VALUES ('오미자차', 8500);""" # VIEW 등록

SQL  = """UPDATE viw_temp               
                 SET    name = '둥글레차'  
                 WHERE  price = 7000 ;"""    # VIEW 수정
SQL_ = """DELETE FROM   viw_temp
                 WHERE  price = 7000 ;;"""   # VIEW 삭제
cur.execute(SQL)
cur.fetchall()  # VIEW를 제작하면 별도로 출력되는 것은 없다

In [184]:
# 3. VIEW 객체 삭제하기
SQL  = """DROP VIEW  viw_temp;"""    # VIEW 삭제
cur.execute(SQL)
cur.fetchall()  # VIEW를 제작하면 별도로 출력되는 것은 없다

[]

In [184]:
# 4. VIEW 삭제 확인하기
SQL  = """DROP VIEW  viw_temp;"""    # VIEW 삭제
cur.execute(SQL)
cur.fetchall()  # VIEW를 제작하면 별도로 출력되는 것은 없다

[]

### <strong>06 집합 연산자
합집합, 교집합, 차집합을 계산

In [191]:
# 수직병합 : 중복되는 자료를 삭제한 나머지를 결합한다
SQL  = """SELECT  name, id     
          FROM    tbl_petlist
          UNION
          SELECT  name, price  
          FROM    tbl_tea;"""
df = pd.read_sql(SQL, con); df

,name,id
0,녹차,1
1,녹차,7000
2,홍차,2
3,홍차,8500


In [194]:
# 수직병합 : 중복되어도 반복하여 결합한다
SQL  = """SELECT  name, id 
          FROM    tbl_petlist
          UNION   ALL
          SELECT  name, price  
          FROM    tbl_tea;"""
df = pd.read_sql(SQL, con); df

,name,id
0,녹차,1
1,홍차,2
2,녹차,7000
3,홍차,8500


### <strong>07 한정 술어
<strong>SubQuery</strong> 결과를 비교조건으로 사용시
1. ALL : 모든값과 비교한다
1. EXISTS : 하나라도 있으면 TRUE를 출력
1. ANY : 아무 값과 일치한다

In [198]:
# SQLite3 에서는 ALL이 Syntex 오류를 발생
SQL  = """SELECT  code, name, price    
          FROM    tbl_tea
          WHERE   price > ALL (SELECT  price 
                               FROM    tbl_tea
                               WHERE   name  LIKE '%녹%');"""
#df = pd.read_sql(SQL, con); df

<br>
## <strong>6 테이블의 구조 바꾸기
### <strong>01 열 구성 변경하기

In [ ]:
# 1. 컬럼의 추가
import sqlite3
con = sqlite3.connect('./sqlite.db')
cur = con.cursor()
SQL = """ALTER  TABLE test             /* 수정할 테이블을 정의 */ 
         ADD          id  VARCHAR(10)  /* 추가컬럼 정의*/ ;""" 
cur.execute(SQL)

In [6]:
# 2. 컬럼의 추가 : 기본값 설정
SQL = """ALTER  TABLE  test                /* 수정할 테이블을 정의 */ 
         ADD           id_val  VARCHAR(10)
         DEFAULT      'test';              /* 추가컬럼 기본값 정의*/""" 
cur.execute(SQL)

In [ ]:
# 3. 컬럼의 삭제는 불가능
# SQLite3 는 Structure의 변경을 어렵게 설계되어 컬럼만 삭제는 불가능하다. )
# 컬럼명을 변경한 새로운 테이블을 생성 후 복사를 한다
# https://blog.xojo.com/2013/12/04/renaming-columns-in-sqlite-tables/
# https://stackoverflow.com/questions/8442147/how-to-delete-or-add-column-in-sqlite
SQL = """ALTER  TABLE   test     /* 수정할 테이블을 정의 */ 
         DROP   COLUMN  id_val  /* 추가컬럼 기본값 정의*/;""" 
# cur.execute(SQL)

### <strong>02 테이블, 열 이름 변경하기
기존의 테이블과 열의 이름을 변경한다

In [ ]:
# 1. 컬럼의 삭제 (SQLite3 는 Structure의 변경을 어렵게 설계되어 컬럼만 삭제는 불가능하다. )
SQL = """ALTER  TABLE   test     /* 수정할 테이블을 정의 */ 
         DROP   COLUMN  id_val  /* 추가컬럼 기본값 정의*/;""" 
# cur.execute(SQL)

In [13]:
# 2. 테이블의 이름 변경
SQL = """ALTER  TABLE  'test' 
         RENAME TO     'stock';"""
cur.execute(SQL)